# Install Dlib

In [ ]:
!pip install dlib

# Fetch Codebase

In [1]:
#@title
import os
os.chdir('/content')
CODE_DIR = 'image_editor'
!git clone https://github.com/raven38/image_edit.git $CODE_DIR
os.chdir(f'./{CODE_DIR}')

FileNotFoundError: [Errno 2] No such file or directory: '/content'

# Define Utility Functions

In [2]:
#@title
import os.path
import io
import IPython.display
import numpy as np
import cv2
from io import BytesIO
from PIL import Image

import torch

from sefa.models import parse_gan_type
from utils import to_tensor
from utils import postprocess
from utils import load_generator
from utils import factorize_weight

def sample(model, gan_type, num=1, codes=None):
    """Samples latent codes."""
    postf = lambda x: x
    if codes is None:
        postf = lambda x: x.detach().cpu().numpy()
        codes = torch.randn(num, model.z_space_dim).cuda()

    if gan_type == 'pggan':
        codes = model.layer0.pixel_norm(codes)
    elif gan_type == 'stylegan':
        codes = model.mapping(codes)['w']
        codes = model.truncation(codes,
                                 trunc_psi=0.7,
                                 trunc_layers=8)
    elif gan_type == 'stylegan2':
        codes = model.mapping(codes)['w']
        codes = model.truncation(codes,
                                 trunc_psi=0.5,
                                 trunc_layers=18)
    return postf(codes)

def synthesize(model, gan_type, code):
    """Synthesizes an image with the give code."""
    if gan_type == 'pggan':
        image = model(to_tensor(code))['image']
    elif gan_type in ['stylegan', 'stylegan2']:
        image = model.synthesis(to_tensor(code))['image']
    image = postprocess(image)[0]
    return image


def make_aligned_image(img_file_buffer, input_image_path, aligned_image_path):
    Image.open(BytesIO(img_file_buffer)).save(input_image_path)
    align_image(tname.name, aligned_image_path)

# Select a Model

In [ ]:
#@title { display-mode: "form", run: "auto" }
model_name = "stylegan_ffhq1024" #@param ['stylegan_ffhq1024', 'pggan_celebahq1024']

generator = load_generator(model_name)
gan_type = parse_gan_type(generator)

# Sample Latent Codes

In [ ]:
#@title { display-mode: "form", run: "auto" }

num_samples = 3 #@param {type:"slider", min:1, max:8, step:1}
noise_seed = 0 #@param {type:"slider", min:0, max:1000, step:1}

codes = sample(generator, gan_type, num_samples, noise_seed)
images = synthesize(generator, gan_type, codes)
imshow(images, col=num_samples)

# Upload Face Image

In [ ]:
#@title
aligned_image = 'align.png'
from google.colab import files
uploaded_files = files.upload()
k, v = list(uploaded_files.items())[0]
make_aligned_image(k, v, aligned_image)
iteration = 500
codes = optimize_style(aligned_image_path, model, model_name, gan_type, base_codes, iteration, bar)

# Factorize & Edit

In [ ]:
#@title { display-mode: "form", run: "auto" }

layer_idx = "0-1" #@param ['all', '0-1', '2-5', '6-13']
semantic_1 = 0 #@param {type:"slider", min:-3.0, max:3.0, step:0.1}
semantic_2 = 0 #@param {type:"slider", min:-3.0, max:3.0, step:0.1}
semantic_3 = 0 #@param {type:"slider", min:-3.0, max:3.0, step:0.1}
semantic_4 = 0 #@param {type:"slider", min:-3.0, max:3.0, step:0.1}
semantic_5 = 0 #@param {type:"slider", min:-3.0, max:3.0, step:0.1}

# Fast implementation to factorize the weight by SeFa.
layers, boundaries, _ = factorize_weight(generator, layer_idx)

new_codes = codes.copy()
for sem_idx in range(5):
  boundary = boundaries[sem_idx:sem_idx + 1]
  step = eval(f'semantic_{sem_idx + 1}')
  if gan_type == 'pggan':
    new_codes += boundary * step
  elif gan_type in ['stylegan', 'stylegan2']:
    new_codes[:, layers, :] += boundary * step
new_images = synthesize(generator, gan_type, new_codes)
imshow(new_images, col=num_samples)